In [ ]:
import airflow
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from trigger_job_util import TriggerJobUtil
from dag_arguments import DAGArgs
import config

def daily_sync_etl():
    try:
        job_name = config.job_name
        temp_location = config.temp_location
        zone = config.zone
        input_data = config.input_data
        input_param_name = config.input_param_name
        gcs_path = config.template_gcs_path
        body = DAGArgs().get_daily_trigger_dataflow_body(job_name,temp_location, zone,input_data,input_param_name)
        TriggerJobUtil.trigger_job(gcs_path, body)
    except Exception as ex:
        print(['Exception', ex])
        
try:
    with DAG(
        'database1_to_database2_daily_sync',
        default_args=DAGArgs().default_args,
        description='DAG for daily sync to cloud sql',
        max_active_runs=1,
        concurrency=4,
        catchup = False,
        schedule_interval='@daily',
        ) as dag:
            task1 = PythonOperator(
                    task_id='daily-sync-etl',
                    python_callable=daily_sync_etl,
                    dag = dag)
except IndexError as ex:
    logging.debug("Exception",str(ex))
    
task1